In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %% load packages
import locale
import sys
import os
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import optuna
import requests
import torch
import random
from sqlalchemy import create_engine,inspect
from pathlib import Path
import urllib.parse
import pyarrow
from calendar import day_abbr
import calendar
from typing import Tuple, Union, Dict, List
from concurrent.futures import ThreadPoolExecutor, as_completed
from pygam import LinearGAM
from datetime import datetime


C:\Users\amaguaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
os.getcwd()

'c:\\Users\\amaguaya\\OneDrive - Kienzle Automotive GmbH\\Desktop\\DLiE\\repo\\DLiE_forecast_13_06_25\\srs\\notebooks'

In [4]:
import sys
sys.path.append('C:/Users/amaguaya/OneDrive - Kienzle Automotive GmbH/Desktop/DLiE/repo/DLiE_forecast_13_06_25')

import os
print("Working directory:", os.getcwd())



Working directory: c:\Users\amaguaya\OneDrive - Kienzle Automotive GmbH\Desktop\DLiE\repo\DLiE_forecast_13_06_25\srs\notebooks


In [5]:
from srs.utils.tutor_utils import prepare_dataset_tensor, forecasting_study,\
  plot_daily_profile,plot_hour_comparison, build_multiwindow_experts, tune_ewa_eta, \
  ewa_aggregate_forecasts, compute_error_table, tune_expert_window, \
  run_expert_window_test, build_regression_matrix, SimpleMLP, train_mlp, \
  prepare_train_test_tensors, build_mlp_rolling_forecasts, tune_mlp_hyperparameters, \
  DST_trafo, prepare_dataset_tensor_modified

from srs.utils.our_utils import run_forecast_step, run_forecast_step_modified
from srs.collect_data.setup import setup_seed, get_device
from srs.collect_data.entsoe_data import create_entsoe_engine, get_tables, get_spec, \
  get_market_divisions,get_map_codes,get_map_codes_starting_with, get_resolution_codes, \
    prepare_generation, prepare_load,prepare_price, prepare_unavailability, \
    prepare_filling_rate_hydro, prepare_physical_flow, prepare_installed_capacity
from srs.collect_data.datastream_data import create_datastream_engine, get_tables, \
  prepare_datastream
from srs.collect_data.dwd_mosmix_data import fetch_region_weather, prepare_weather
from srs.collect_data.merge_data import merge_datasets, build_training_dataset



### models_24h fitting for all no1-no5 regions separately.

In [6]:
'''
  eval/test periods
  2023 - 365 days
  2024 - 366 days
'''
# rmse  # -- 31.68, rmse: 8.79, 8.798 with rolling, 8.77 use n+180, -90... 8.7978; -120 (4months):9.51 
# -30 (1 month)8.72, -60 (2mothns), 8.703, if remove 2019. 8.73

# id_init_eval-1
# data_t.shape, train_t.shape, train_dates.shape
# 2192-751

# len(price_t[24:-24])
# len(data_t[:,:,0])

'\n  eval/test periods\n  2023 - 365 days\n  2024 - 366 days\n'

In [7]:
# len(price_t[:-1])
# len(full_dates_), len(data_t[:,:,0].cpu().numpy())

In [ ]:
# # Step 1: Flatten the time series (days × hours → 1D)
# flattened_series = data_t[:, :, 0].reshape(-1).cpu().numpy()

# # Step 2: Create a matching hourly datetime index
# # There are 24 hours per day, so total points = 2192 days × 24 = 52608
# start_datetime = pd.to_datetime(INIT_DATE_EXPERIMENTS)
# end_datetime = pd.to_datetime(FINAL_DATE_EXPERIMENTS) + pd.Timedelta(hours=23)
# datetime_index = pd.date_range(start=start_datetime, end=end_datetime, freq='H')

# # Check: both must match in length
# assert len(flattened_series) == len(datetime_index)

# # Step 3: Create the Series and plot
# price_series_flat = pd.Series(flattened_series, index=datetime_index)

# plt.figure(figsize=(15, 4))
# price_series_flat[-(24*7*0)-(2*24):].plot()
# plt.title("Electricity Price (Hourly)")
# plt.xlabel("Time")
# plt.ylabel("Price")
# plt.grid(True)
# plt.tight_layout()
# plt.show()

# Create mask: keep only valid forecast rows (no NaNs across the whole 24h horizon)
# valid_mask = ~torch.isnan(forecasts_zone[:, :, 0]).any(dim=1)
# # Apply mask to both forecast and true values
# valid_forecast = forecasts_zone[valid_mask, :, 0]
# valid_true     = true_vals[valid_mask, :]
# # Compute RMSE only on valid rows
# diff = valid_forecast - valid_true
# rmse = torch.sqrt((diff ** 2).mean()).item()
# print(f"Filtered RMSE: {rmse:.4f}")


Filtered RMSE: 9.4453


In [8]:
# set the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# **************************************
# define dates for training and evaluation 
# **************************************
INIT_DATE_EXPERIMENTS = '2019-01-01'
INIT_TEST_DATE = '2023-01-01'
FINAL_DATE_EXPERIMENTS = '2024-12-31'
n_days_test = (pd.to_datetime(FINAL_DATE_EXPERIMENTS) - pd.to_datetime(INIT_TEST_DATE)).days + (1) # additional adjustment

repo_root = Path.cwd().parents[1]
mapcodes = ["NO1"] # ["NO1","NO2","NO3","NO4","NO5"]
maps_dict = {}

for code in mapcodes:
    csv_path = repo_root / "data" / f"data_{code}.csv"
    df = pd.read_csv(csv_path, parse_dates=["time_utc"])
    data_t, train_t, train_dates, price_t = prepare_dataset_tensor_modified(
        csv_path,
        tz="CET",
        seed=42,
        test_days= n_days_test , # Here it should be 1*365 for 1 year of test, and so on
        dtype=torch.float64,
    )

    # fix potential problems with dates after change time zone..
    train_dates_series       = pd.DatetimeIndex(sorted(train_dates))
    id_init_exp = train_dates_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_test_period = train_dates_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_exp = train_dates_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    data_t = data_t[id_init_exp:(id_end_exp+1), :,:]
    train_dates = pd.Series(train_dates[id_init_exp:(id_end_exp+1)])

    print(f"days for test data ..{data_t.shape[0] - id_init_test_period}")
    
    maps_dict[code] = {
        "df": df,
        "data_t": data_t,
        "train_t": train_t,
        "train_dates": train_dates,
        "price_t": price_t
    }
maps_dict.keys()

lgbm24_by_zone   = {}
rmse_by_zone  = {}



for z in mapcodes:
    print(f"\n--- {z} ---")
    price_S      = maps_dict[z]["price_t"]
    data_array   = maps_dict[z]["data_t"]
    full_dates      = maps_dict[z]["train_dates"] # <-- here i change a bit the previuous function, prepare_data_tensor, it has all dates
    feature_names= maps_dict[z]["df"].columns[1:]
    full_date_series = pd.DatetimeIndex(sorted(full_dates))

    # evaluation days (all of 2024)
    train_start_idx = full_date_series.get_loc(pd.Timestamp(INIT_DATE_EXPERIMENTS))
    id_init_eval = full_date_series.get_loc(pd.Timestamp(INIT_TEST_DATE))
    id_end_eval = full_date_series.get_loc(pd.Timestamp(FINAL_DATE_EXPERIMENTS))
    eval_start_idx = id_init_eval
    eval_end_idx  = id_end_eval
    N_s            = (eval_end_idx - eval_start_idx) + 1 # additional adjustment
    full_dates = pd.to_datetime(full_dates)

    

    # new features: WD - dummy for week days, price lags for Mon, Tue and Fri, day-ahead load lag
    WD             = [1,6,7]     
    PRICE_S_LAGS   = [1,2,7]
    DA_LAG         = [0]
    S              = 24
    #D             = 730

    # tensors to collect forecasts for THIS zone
    forecasts_zone = torch.full((N_s, S, 1), float("nan"),
                                dtype=torch.float64, device=device)

    # thread pool
    with ThreadPoolExecutor(max(1, os.cpu_count() // 2) + 1 ) as executor:
        futures = [
            executor.submit(
                run_forecast_step_modified,
                n,
                price_S,
                data_array,
                train_start_idx = train_start_idx,
                train_end_idx   = id_init_eval - 1,
                full_dates      = full_dates,
                wd              = WD,                
                price_s_lags    = PRICE_S_LAGS,
                da_lag          = DA_LAG,
                feature_names   = feature_names,   # reg_names
                n_trials_lgbm   = 10,
                days_for_st_model = 7
            )
            for n in range(N_s)[:10]
        ]

        for fut in as_completed(futures):
            try:
                n, lgbm24 = fut.result()
                forecasts_zone[n, :, 0] = lgbm24.detach().clone().to(forecasts_zone.dtype).to(forecasts_zone.device)
            except Exception as e:
                import traceback
                print(f"Thread crashed: {e}")
                traceback.print_exc()

                
    #   shape: (N_s, S)
    true_vals = price_S[eval_start_idx : eval_end_idx + 1].to(device)  
    
    # compute RMSE
    diff = forecasts_zone[:, :, 0] - true_vals
    rmse = torch.sqrt((diff**2).mean()).item()
    
    print(f"Zone {z} LBGM-24h RMSE: {rmse:.4f}")

    lgbm24_by_zone[z] = forecasts_zone[:, :, 0].cpu()
    rmse_by_zone[z]  = rmse

[I 2025-06-13 20:46:46,507] A new study created in memory with name: no-name-131f1866-f8e0-4d68-8fc0-8947094cf679
[I 2025-06-13 20:46:46,509] A new study created in memory with name: no-name-fca073d0-6a4b-4c06-bbb7-bf6f340b6754
[I 2025-06-13 20:46:46,510] A new study created in memory with name: no-name-ef4ade62-dc35-495f-973b-f6a10b3256bc
[I 2025-06-13 20:46:46,514] A new study created in memory with name: no-name-0c8e4e9a-2ff0-4df6-8039-173e97ae51fa
[I 2025-06-13 20:46:46,520] A new study created in memory with name: no-name-9e6fb42a-b162-469e-83a5-37980b5f41ea
[I 2025-06-13 20:46:46,527] A new study created in memory with name: no-name-e766ad48-901c-4d77-a6e8-ea3beddd9401
[I 2025-06-13 20:46:46,542] A new study created in memory with name: no-name-aa70a7c4-6909-4081-a081-cdeb9cb1d897


days for test data ..731

--- NO1 ---
Loop   0: train 2019-01-01 00:00:00 -> 2022-12-31 00:00:00, forecast 2023-01-01 00:00:00
  dat_slice shape: torch.Size([1462, 24, 10])  → flatten count = 35088
Loop   1: train 2019-03-03 00:00:00 -> 2023-01-01 00:00:00, forecast 2023-01-02 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   2: train 2019-03-04 00:00:00 -> 2023-01-02 00:00:00, forecast 2023-01-03 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   3: train 2019-03-05 00:00:00 -> 2023-01-03 00:00:00, forecast 2023-01-04 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   4: train 2019-03-06 00:00:00 -> 2023-01-04 00:00:00, forecast 2023-01-05 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Loop   5: train 2019-03-07 00:00:00 -> 2023-01-05 00:00:00, forecast 2023-01-06 00:00:00
  dat_slice shape: torch.Size([1402, 24, 10])  → flatten count = 33648
Lo

[I 2025-06-13 20:46:48,274] Trial 0 finished with value: 0.008735058602127187 and parameters: {'n_estimators': 226, 'learning_rate': 0.014938652247999019, 'num_leaves': 43, 'max_depth': 4, 'min_child_samples': 11, 'subsample': 0.9378786624814134, 'colsample_bytree': 0.8742119880677611, 'reg_alpha': 2.1671045092255, 'reg_lambda': 9.217328469985437, 'min_split_gain': 0.0007355400884160091}. Best is trial 0 with value: 0.008735058602127187.
[I 2025-06-13 20:46:48,386] Trial 0 finished with value: 0.006682397042725551 and parameters: {'n_estimators': 209, 'learning_rate': 0.0719856452768991, 'num_leaves': 104, 'max_depth': 11, 'min_child_samples': 29, 'subsample': 0.8287003680067984, 'colsample_bytree': 0.7634835221989527, 'reg_alpha': 8.188494538281446, 'reg_lambda': 9.64353014497456, 'min_split_gain': 0.026935788147820647}. Best is trial 0 with value: 0.006682397042725551.


[Trial 0] New best: 0.00874
[Trial 0] New best: 0.00668


[I 2025-06-13 20:46:49,036] Trial 0 finished with value: 0.005655017958577709 and parameters: {'n_estimators': 171, 'learning_rate': 0.08181557635629697, 'num_leaves': 231, 'max_depth': 5, 'min_child_samples': 29, 'subsample': 0.8427809775189803, 'colsample_bytree': 0.5346151590333711, 'reg_alpha': 0.5854079741219009, 'reg_lambda': 1.7216885631847478, 'min_split_gain': 0.008345528238898964}. Best is trial 0 with value: 0.005655017958577709.
[I 2025-06-13 20:46:49,097] Trial 0 finished with value: 0.01835938258619427 and parameters: {'n_estimators': 109, 'learning_rate': 0.016536145810672413, 'num_leaves': 206, 'max_depth': 12, 'min_child_samples': 12, 'subsample': 0.7943790573617941, 'colsample_bytree': 0.6333404615228864, 'reg_alpha': 4.885064561407814, 'reg_lambda': 7.763469672117815, 'min_split_gain': 0.020451710164683685}. Best is trial 0 with value: 0.01835938258619427.


...

[Trial 6] No significant gain (Δ ≤ 0.001). Counter: 2/3
long model ************************
             feature  importance
0        Price_lag_1         950
1        Price_lag_2         839
3      Load_DA_lag_0         545
2        Price_lag_7         411
11        NGas_lag_2         312
10        Coal_lag_2         286
5    WindOn_DA_lag_0         235
12         Oil_lag_2         232
13         EUA_lag_2         168
9               WD_7           5
8               WD_6           1
4     Solar_DA_lag_0           0
6   WindOff_DA_lag_0           0
7               WD_1           0
short model ************************
             feature  importance
0        Price_lag_1         109
3      Load_DA_lag_0          21
1        Price_lag_2           8
13         EUA_lag_2           1
2        Price_lag_7           0
4     Solar_DA_lag_0           0
5    WindOn_DA_lag_0           0
6   WindOff_DA_lag_0           0
7               WD_1           0
8               WD_6           0
9            

[I 2025-06-13 20:47:20,157] Trial 7 finished with value: 0.01890528957787868 and parameters: {'n_estimators': 134, 'learning_rate': 0.18023606238283535, 'num_leaves': 194, 'max_depth': 5, 'min_child_samples': 13, 'subsample': 0.7747217135616025, 'colsample_bytree': 0.8515485777892962, 'reg_alpha': 9.999042562296909, 'reg_lambda': 4.897750962564925, 'min_split_gain': 0.033933559903182184}. Best is trial 4 with value: 0.013497513060122937.
[I 2025-06-13 20:47:20,159] A new study created in memory with name: no-name-f2be1c13-78cf-49a4-abef-253c6e22f1af
[I 2025-06-13 20:47:20,258] Trial 0 finished with value: 0.25033680535531916 and parameters: {'n_estimators': 254, 'learning_rate': 0.036842332607035885, 'num_leaves': 42, 'max_depth': 7, 'min_child_samples': 16, 'subsample': 0.6470959404364198, 'colsample_bytree': 0.7937480073541355, 'reg_alpha': 8.470022886532446, 'reg_lambda': 4.043023100630952, 'min_split_gain': 0.009049924567393458}. Best is trial 0 with value: 0.25033680535531916.


[Trial 7] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.


[I 2025-06-13 20:47:20,400] Trial 4 finished with value: 0.026331061961006135 and parameters: {'n_estimators': 183, 'learning_rate': 0.01152468685335159, 'num_leaves': 132, 'max_depth': 11, 'min_child_samples': 2, 'subsample': 0.6239269918579105, 'colsample_bytree': 0.5898679866216323, 'reg_alpha': 8.245828038804474, 'reg_lambda': 5.962619920570765, 'min_split_gain': 0.01494997051134791}. Best is trial 3 with value: 0.01017231061059982.
[I 2025-06-13 20:47:20,402] A new study created in memory with name: no-name-e473e9a2-fba9-4314-9cb8-b4723a3a4905
[I 2025-06-13 20:47:20,516] Trial 0 finished with value: 0.28200804675364016 and parameters: {'n_estimators': 117, 'learning_rate': 0.012252476390887302, 'num_leaves': 48, 'max_depth': 13, 'min_child_samples': 16, 'subsample': 0.7348059109646637, 'colsample_bytree': 0.7097079999061746, 'reg_alpha': 3.8068261481911936, 'reg_lambda': 7.797497571513462, 'min_split_gain': 0.013252985983098459}. Best is trial 0 with value: 0.28200804675364016.


[Trial 4] No significant gain (Δ ≤ 0.001). Counter: 3/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
[Trial 0] No significant gain (Δ ≤ 0.001). Counter: 4/3

⏹️  Early stop: no gain > 0.001 in last 3 trials.
long model ************************
             feature  importance
0        Price_lag_1        3383
1        Price_lag_2        2901
2        Price_lag_7        1331
11        NGas_lag_2        1331
10        Coal_lag_2        1232
3      Load_DA_lag_0         821
12         Oil_lag_2         560
5    WindOn_DA_lag_0         489
13         EUA_lag_2         448
9               WD_7           7
4     Solar_DA_lag_0           0
6   WindOff_DA_lag_0           0
7               WD_1           0
8               WD_6           0
short model ************************
             feature  importance
0        Price_lag_1         193
1        Price_lag_2          19
3      Load_DA_lag_0          13
11        NGas_lag_2           3
2        Price_lag_7           0
4     Solar_DA_lag_0